1. First, do some data preprocessing to clean up the data as necessary. You can use your solution to the assignment of the Text preprocessing checkpoint.


In [1]:
import pandas as pd
import numpy as np
import nltk
import spacy
import gensim
import re
import random
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings(action="ignore")
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 7.6 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [23]:
pip install markovify

     |████████████████████████████████| 235 kB 12.8 MB/s 
  Created wheel for markovify: filename=markovify-0.9.3-py3-none-any.whl size=18622 sha256=6d4a1bcadc840bbab469592c870f769af2fa1bf4d9f613c4ba32f4c85d7628fc
  Stored in directory: /root/.cache/pip/wheels/d9/f0/5b/748a27bdf2496bd4df51acb9442dae516efce507ff4849813e
Successfully built markovify


In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'twitter_sentiment'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

twitter_df = pd.read_sql_query('select * from twitter',con=engine)

engine.dispose()

In [6]:
twitter_df.head()

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


In [16]:
# Utility function for standard text cleaning
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation that spaCy doesn't
    # recognize: the double dash --.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = ' '.join(text.split())
    return text

    twitter_df = text_cleaner(twitter_df)

In [18]:
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 20000000
twitter_doc = nlp(" ".join(twitter_df.text))


2. Train a Markov chain model by using only the negative sentiment tweets. Generate some random sentences. Do the generated sentences exhibit the same negative sentiment?

In [19]:
twitter_doc_negative = nlp(" ".join(twitter_df[twitter_df.airline_sentiment=='negative'].text))


In [30]:
twitter_negative_sents = [sent.text for sent in twitter_doc_negative.sents if len(sent.text) > 1]


In [28]:
import markovify
Tweet_negative_generator = markovify.Text(twitter_negative_sents, state_size = 3)

In [29]:
# Print 4 randomly generated sentences
for i in range(4):
    print(Tweet_negative_generator.make_sentence())

None
@AmericanAir customer service is poor.
You gotta work on your online checkout system.
None


In [36]:
class POSifiedText(markovify.Text):
    
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

In [26]:
Tweet_negative_generator = POSifiedText(twitter_negative_sents, state_size = 3)

In [27]:
# Print 4 randomly generated sentences
for i in range(4):
    print(Tweet_negative_generator.make_sentence())

Now holding breath while they keep me trapped 3 hours to de - ice 5 minutes after I sent it to you .
Not happy standing in a line waiting for 40 minutes again .
@SouthwestAir well , it was pretty full so we had to find out my options , yet no response .
grumpykim @united Never had a flight that was Cancelled Flighted Saturday .


Sentences show negative sentiment but some of them are not meaning full.

3. Repeat the previous task, but this time, use only the positive sentiment tweets. Generate some random sentences and observe whether they exhibit positive sentiment or not.


In [31]:
twitter_doc_positive = nlp(" ".join(twitter_df[twitter_df.airline_sentiment=='positive'].text))

In [32]:
twitter_positive_sents = [sent.text for sent in twitter_doc_positive.sents if len(sent.text) > 1]

In [33]:
Tweet_positive_generator = markovify.Text(twitter_positive_sents, state_size = 3)

In [35]:
# Print 4 randomly generated sentences
for i in range(4):
    print(Tweet_positive_generator.make_sentence())

I got thru on the phone did a great job.
Only 8 minutes to get my bag to me.
@JetBlue thanks to Julian at login for getting me to Orlando early #happiness @JetBlue flight from JFK-SFO was pure awesomeness.
None


In [37]:
Tweet_positive_generator = POSifiedText(twitter_positive_sents, state_size = 3)

In [38]:
# Print 4 randomly generated sentences
for i in range(4):
    print(Tweet_positive_generator.make_sentence())

😄 @JetBlue Big thanks to your team .
None
@JetBlue offers hot tea and coffee ... at the gate at MSY went above and beyond to help me get the boys & amp ; free .
@united thank you @united finally got through the second time I called ,   thanks for the info !


All sentences show negative sentiment.

4. Now, train your Markov chain model using all of the tweets. Generate some random sentences. Can you say something systematic about the sentiments of the generated tweets?

In [39]:
twitter_doc = nlp(" ".join(twitter_df.text))

In [40]:
twitter_sents = [sent.text for sent in twitter_doc.sents if len(sent.text) > 1]

In [41]:
Tweet_generator = markovify.Text(twitter_sents, state_size = 3)

In [42]:
# Print 4 randomly generated sentences
for i in range(4):
    print(Tweet_generator.make_sentence())

I like the quick response - it's appreciated!
jetblue http://t.co/HDoXiM6NZ2 @JetBlue once you go blue you don't go back @JetBlue why are you letting us go to the airport &amp; got to speak to someone on the plane yesterday.
None
You've Cancelled Flightled 3 times, now leaving 72 hours Late Flight but that's better than being Cancelled Flightled!


In [44]:
Tweet_generator = POSifiedText(twitter_sents, state_size = 3)

In [45]:
# Print 4 randomly generated sentences
for i in range(4):
    print(Tweet_generator.make_sentence())

@AmericanAir seriously , there are mechanical problems .
This is the same passenger , just her maiden name ... married 50 ... no docs with maiden name .
Screw you @AmericanAir @AmericanAir can I DM ?
@united at the gate , thanks for the 2hr delay and then no ground crew when we landed , instead letting us wait 45 minutes with no human interaction .


This time both negative and positive sentiments can be seen.